In [10]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from fredapi import Fred
from utils import crosscorr

## Отталкиваемся от GDP, так как у этого временного ряда самый широкий существующий диапазон значений 

In [6]:
# сюда вставить свой токен с https://fred.stlouisfed.org/docs/api/api_key.html
api_key = '2f65863001e3895e413ea46a51d481d7'
base_series = 'GDP'
fred = Fred(api_key = api_key)
data = fred.get_series(base_series)
data = pd.DataFrame({'date': data.index, base_series: data.values})
data.tail(5)

,date,GDP
301,2021-04-01,23046.934
302,2021-07-01,23550.420
303,2021-10-01,24349.121
304,2022-01-01,24740.480
305,2022-04-01,25248.476


## Далее клеим что хотим

In [7]:
series_dict = {
    # сюда вставлять ticker с fred-а и произвольную расшифровку для понимания
    "PPIACO": "Producer Price Index by Commodity: All Commodities", 
    "CORESTICKM159SFRBATL": "Sticky Price Consumer Price Index less Food and Energy",
    "PCE": "Personal Consumption Expenditures", # Consumption
    "GPDI": "Gross Private Domestic Investment" # Investment
    
}
for s in series_dict.keys():
    series_data = fred.get_series(s)
    series_data = pd.DataFrame({'date':series_data.index, s: series_data.values})
    data = data.merge(series_data, how = "left", on = "date")
data.tail(10)
    
    

,date,GDP,PPIACO,CORESTICKM159SFRBATL,PCE,GPDI
296,2020-01-01,21538.032,199.300,2.788599,14728.8,3737.555
297,2020-04-01,19636.731,185.500,2.233555,12082.4,3161.422
298,2020-07-01,21362.428,193.000,2.350691,14230.1,3743.278
299,2020-10-01,21704.706,196.500,1.848308,14626.1,3929.446
300,2021-01-01,22313.850,204.800,1.504532,14932.2,3902.314
301,2021-04-01,23046.934,217.900,2.284027,15712.6,3943.400
302,2021-07-01,23550.420,231.850,2.325734,16027.9,4109.122
303,2021-10-01,24349.121,240.465,3.042940,16473.7,4499.171
304,2022-01-01,24740.480,246.453,3.982988,16725.6,4671.027
305,2022-04-01,25248.476,265.310,4.723538,17115.6,4609.927


In [11]:
date_sample = data[
    (data["date"]>="1970-01-01") & 
    (data["date"]<="2020-12-31") &
    # отбираем года, поэтому за каждый год берем январь
    (data["date"].dt.month == 1)
]

In [12]:
date_sample.tail(3)

,date,GDP,PPIACO,CORESTICKM159SFRBATL,PCE,GPDI
288,2018-01-01,20155.486,197.9,2.166829,13628.4,3551.123
292,2019-01-01,21013.085,199.1,2.378344,14090.0,3778.820
296,2020-01-01,21538.032,199.3,2.788599,14728.8,3737.555


In [13]:
def build_correlations(date_sample: pd.DataFrame, first_ticker: str, second_ticker: str):
    corrs = []
    for lag in list(range(-4, 5)):
        corr = crosscorr(date_sample[first_ticker], date_sample[second_ticker], lag)
        corrs.append([lag, round(corr, 3)])
    return pd.DataFrame(corrs, columns=["lag", "correlation"])



In [14]:
gdp_gdpi = build_correlations(date_sample, "GDP", "GPDI")


In [16]:
gdp_gdpi.T

,0,1,2,3,4,5,6,7,8
lag,-4.000,-3.000,-2.00,-1.000,0.000,1.000,2.000,3.000,4.000
correlation,0.975,0.976,0.98,0.985,0.988,0.987,0.985,0.982,0.979
